In [2]:
import pandas as pd
import numpy as np

In [5]:
# import data
data = pd.read_csv('/Users/jimliu/Learning/Kaggle/House Price Predicting /all/train.csv')

# Check out percentage of missing values

In [33]:
data.describe()


# check out missings

missing = data.isnull().sum()
x = missing.loc[missing > 0]/len(data)

# variables would be delted
del_vars = list(x.loc[x > 0.5].index)

['Alley', 'PoolQC', 'Fence', 'MiscFeature']